In [1]:
import numpy as np

In [ ]:
class SensoryNeuron:
    
    def __init__(
        self,
        eye,
        orientation,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_adaptation,
        init_activity=None
    ):
        self.eye = eye
        self.orientation = orientation
        self.alpha = alpha
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_adaptation = weight_adaptation
        if init_activity is None:
            self.mean_response = np.random.rand()
        else:
            self.mean_response = init_activity
        
    def 
        
    def excitatory_drive(
        self,
        sensory_input
    ):
        pass
    
    def supressive_drive(
        self
    ):
        pass
    
    def change_habituation(
        self
    ):
        pass

In [ ]:
class AttentionNeuron:
    def __init__(
        self,
        
    ):
        pass

In [9]:
import os

0

In [ ]:
sensory_layer = [SensoryNeuron('right', 1), SensoryNeuron('...')]
attention_layer = [...]

#define input
condition = 'switch_stimuli'
stimulus_input = get_input(condition)

def get_input(condition, n_trials):
    if condition == 'switch_stimuli':
        return np.array([0,1] * (n_trials // 2))
    else:
        raise 
        
#init_firing_rate
class Snapshot:
    def __init__(self, layers):
        ...
def extract_results(snapshots):
    pass

snaps = []
t_steps = 10000
t_delta = .01
for i in range(t_steps):
    current_time = t_delta * i
    c_input = stimulus_input[i]
    snap.append(Snapshot(sensory_layer, attention_Layer, opponency_layer))
    sensory_layer.update_weights(c_input, snap[-1])
    attention_layer.update_weights(snap[-1])
    opponency_layer.update_weights(snap[-1])
    
results = extract_results(snap)